# Find sites where we have detailed observations of density over all depths. One high and one low accumulation

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sumup = xr.open_dataset("sumup_density_2020.nc")
su_date = sumup['Date'].values
su_lat = sumup['Latitude'].values
su_lon = sumup['Longitude'].values
su_elevation = sumup['Elevation'].values
su_citation = sumup['Citation'].values
su_density = sumup['Density'].values
su_midpoint = sumup['Midpoint'].values
su_citation = sumup['Citation'].values

In [3]:
sumup

<xarray.Dataset>
Dimensions:      (Citation: 2283706, Date: 2283706, Dens: 2283706, Elevation: 2283706, Error: 2283706, Lat: 2283706, Lon: 2283706, Method: 2283706, Midpoint: 2283706, SDOS_Flag: 2283706, Start_Depth: 2283706, Stop_Depth: 2283706)
Coordinates:
  * Date         (Date) float32 20040000.0 20040000.0 ... 20181210.0 20181210.0
  * Start_Depth  (Start_Depth) float32 -9999.0 -9999.0 -9999.0 ... 0.8 0.9 1.0
  * Stop_Depth   (Stop_Depth) float32 -9999.0 -9999.0 -9999.0 ... 0.9 1.0 1.1
  * Midpoint     (Midpoint) float32 0.075 0.105 0.135 0.165 ... 0.85 0.95 1.05
  * Error        (Error) float32 -9999.0 -9999.0 -9999.0 ... -9999.0 -9999.0
  * Elevation    (Elevation) float32 -9999.0 -9999.0 -9999.0 ... -9999.0 -9999.0
  * SDOS_Flag    (SDOS_Flag) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
  * Method       (Method) float32 3.0 3.0 3.0 3.0 3.0 ... 1.0 1.0 1.0 1.0 1.0
  * Citation     (Citation) float32 9.0 9.0 9.0 9.0 ... 196.0 196.0 196.0 196.0
Dimensions without coordinates: Dens, Lat, Lon
Data variables:
    Latitude     (Lat) float32 -80.7901 -80.7901 ... -70.80362 -70.80362
    Longitude    (Lon) float32 124.4345 124.4345 ... -71.714775 -71.714775
    Density      (Dens) float32 0.48 0.44 0.39 0.4 ... 344.0 385.0 370.0 402.0

In [4]:
# Define Filter 
filter_func = np.vectorize(lambda lat, elevation: (lat > -91 and lat < -45 and elevation > 0))

# Execute filter
indices = filter_func(su_lat, su_elevation)
lat = su_lat[indices]
lon = su_lon[indices]
dens = su_density[indices]
mp = su_midpoint[indices]

# Merge vectors
lat_lon = np.transpose(np.array([lat, lon]))

# Find unique lat/lon pairs
lat_lon = np.unique(lat_lon, axis=0)
n_sumup = len(lat_lon)
print(n_sumup)

842


In [4]:
# User settings: search parameters
lat_min   = -91   # From South Pole
lat_max   = -45   # To 45 S
depth_min = 1
depth_max = 30
exclude_ice_shelves = True  # Exclude ice shelves from search yes/no
write_profiles = False   # Write profiles in "./profiles/" directory yes/no

# Define Filter 
filter_func = np.vectorize(lambda lat, elevation: (lat > lat_min and lat < lat_max and (not exclude_ice_shelves or elevation > 0)))

# Execute filter
indices = filter_func(su_lat, su_elevation)
date = su_date[indices]
lat = su_lat[indices]
lon = su_lon[indices]
dens = su_density[indices]
mp = su_midpoint[indices]
cit = su_citation[indices]

# Merge vectors
lat_lon = np.transpose(np.array([lat, lon]))

# Find unique lat/lon pairs
lat_lon = np.unique(lat_lon, axis=0)




# Helper variables for the search
hasShallow = False # Tests for depth_min
hasDeep = False    # Tests for depth_max
slat = -1
slon = -1
sdate = -1
jtop = -1
jbottom = -1


for j in range(0, len(mp)):
    # Check if it is the first site, or if we arrive at a new site
    if (slat == -1 or slat != lat[j] or slon == -1 or slon != lon[j]):
        if (hasShallow and hasDeep):
            # If the site has both shallow and deep data points, write files containing the density profiles
            print(sdate, scit, jtop, jbottom, (jbottom - jtop), slat, slon)
            if (write_profiles):
                fn = "./profiles/" + str(int(sdate)) + "_" + str(slat) + "_" + str(slon) + ".dat"
                f = open(fn, "w")
                for i in range(jtop, jbottom+1):
                    f.write(str(mp[i]) + " " + str(dens[i]*1000.) + "\n")
                f.close()
        # Set new site, and reset helper variables
        slat = lat[j]
        slon = lon[j]
        sdate = date[j]
        scit = cit[j]
        hasShallow = False
        hasDeep = False
        jtop = j
        jbottom = j
    # Check for shallow:
    if (mp[j] <= depth_min):
        hasShallow = True
    # Check for deep:
    if (mp[j] >= depth_max):
        hasDeep = True
    # Update the bottom index
    jbottom = j

20101222.0 7.0 2167 2227 60 -77.957 -95.962
20110114.0 7.0 2228 2305 77 -76.952 -121.22
20101232.0 7.0 2306 2425 119 -76.77 -101.738
19971220.0 122.0 61833 63343 1510 -75.5815 -3.430333
19971224.0 140.0 74233 76611 2378 -75.002335 0.007
19971224.0 145.0 82951 87914 4963 -75.167336 -0.995
19980108.0 155.0 95076 96046 970 -75.167 6.4985
19980104.0 159.0 109696 115582 5886 -74.66717 4.001667
19980102.0 163.0 115611 122531 6920 -74.49917 1.960833
